# SST Individual Discriminability

This notebook measures discriminability of conditions for each subject, within runs.

It then takes the average discriminability across any multiple runs (I don't think we have them though).

Then we can compare discriminability against other subject-level variables.



Something similar was previously been done in `rsa_within_subj.ipynb`; it took similarity between matrices representing the averages of GNG and compared them intra-class similarity.

That file doesn't take the next step of trying to see how that similarity score might measure up against across-subject variables; it seems, because I did not find good evidence that intrasimilarity was actually higher than interclass similarity.

Conclusions:

 - maybe I should have tried discriminability, not just similarity
 - If we're interested in discriminability, then running an ML algorithm should be superior?
 - But don't rule out going back to similarity.

### General set-up

## TESQ

### TESQ Machine learning set-up

Some of the below can probably be moved to "general set-up" as appropriate.

In [1]:
from discriminability_tools import *

python initialized for apply_loocv_and_save


/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/datasets/__init__.py:87: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


28 25
28


In [2]:
import pickle
from IPython.core.display import display, HTML, Markdown
from nilearn.decoding import Decoder
from sklearn.model_selection import StratifiedKFold
from random import randint
import math

In [3]:
import sys
import os
import pandas as pd
import gc

sys.path.append(os.path.abspath("../../ml/"))

from apply_loocv_and_save import *
from dev_wtp_io_utils import *
import gc
import nibabel as nib

from os import path



nonbids_data_path = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/"
ml_data_folderpath = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml"
train_test_markers_filepath = ml_data_folderpath + "/train_test_markers_20210601T183243.csv"
test_train_df = pd.read_csv(train_test_markers_filepath)

all_sst_events= pd.read_csv(ml_data_folderpath +"/SST/" + "all_sst_events.csv")


dataset_name = 'conditions'

from nilearn.decoding import DecoderRegressor, Decoder

script_path = '/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml'
# HRF 2s

#get a PFC mask
#pfc_mask = create_mask_from_images(get_pfc_image_filepaths(ml_data_folderpath + "/"),threshold=10)


def trialtype_resp_trans_func(X):
    return(X.trial_type)



In [4]:

cpus_available = int(os.getenv('CPUS_PER_TASK'))
#custom thing I have set in my jupyter notebook task.
print(cpus_available)
cpus_to_use = cpus_available-1

4


### TESQ Analysis set-up

In [5]:
from analyze_results import remove_selected_outliers
from scipy.stats import pearsonr,spearmanr
from matplotlib import pyplot

def remove_selected_outliers_tesq_study(ind_div_combined,show_plot=False):
    idc_outliers_removed = remove_selected_outliers(ind_div_combined,
    ['discriminability_overfit_accuracy','discriminability_mean_cv_scores',
        'BFI_extraversion','RMQ_locomotion','ses_aggregate','PLAN_cognitive_strategies',
     'SST_SSRT','BIS_11','BSCS','TESQ_E_suppression', 'TESQ_E_avoidance_of_temptations', 
     'TESQ_E_goal_deliberation', 'TESQ_E_controlling_temptations', 'TESQ_E_distraction',
     'TESQ_E_goal_and_rule_setting','EDM','RS','TRSQ','ROC_Crave_Regulate_Minus_Look',
     'SRHI_unhealthy',
     'cancer_promoting_minus_preventing_FFQ','bf_1'],
    show_plot=show_plot)
    return(idc_outliers_removed)

def display_discriminability_correlations(ind_div_combined_3sd):
    for neural_var in ['discriminability_overfit_accuracy','discriminability_mean_cv_scores']:
        display(Markdown("### " + neural_var))
        for correlate in ['BFI_extraversion','RMQ_locomotion','ses_aggregate','PLAN_cognitive_strategies',
                          'SST_SSRT','BIS_11','BSCS',
                          'TESQ_E_suppression', 'TESQ_E_avoidance_of_temptations', 
                          'TESQ_E_goal_deliberation', 'TESQ_E_controlling_temptations', 'TESQ_E_distraction',
                          'TESQ_E_goal_and_rule_setting',
                        'EDM','RS','TRSQ','ROC_Crave_Regulate_Minus_Look','SRHI_unhealthy']:
            display(Markdown("#### " + correlate))
            nan_rows = np.isnan(ind_div_combined_3sd[correlate]) | np.isnan(ind_div_combined_3sd[neural_var])
            cor2way_df = ind_div_combined_3sd.loc[nan_rows==False,]
            pearson_result = pearsonr(cor2way_df[neural_var],cor2way_df[correlate])
            display(HTML("r=" + format(pearson_result[0],".2f") +"; p-value=" + format(pearson_result[1],".4f")))
            spearman_result = spearmanr(cor2way_df[neural_var],cor2way_df[correlate])
            display(HTML("rho=" + format(spearman_result[0],".2f") +"; p-value=" + format(spearman_result[1],".4f")))
            cplot = pyplot.scatter(cor2way_df[neural_var],cor2way_df[correlate])
            cplot.axes.set_xlabel(neural_var)
            cplot.axes.ylabel=correlate
            pyplot.show()

## pfc only

In [6]:
dataset_name = 'conditions'


brain_data_filepath = ml_data_folderpath + '/SST/Brain_Data_betaseries_84subs_correct_cond.pkl'
#brain_data_filepath = ml_data_folderpath + '/SST/Brain_Data_betaseries_58subs_correct_cond.pkl'
#brain_data_filepath = ml_data_folderpath + '/SST/Brain_Data_conditions_43subs_correct_cond.pkl'

# def decoderConstructor(*args, **kwargs):
#     return(Decoder(scoring='accuracy',verbose=0, *args, **kwargs))

In [7]:
pfc_mask = create_mask_from_images(get_pfc_image_filepaths(ml_data_folderpath + "/"),threshold=10)
relevant_mask = pfc_mask


(8, 238955)
2


In [8]:
all_subjects = load_and_preprocess(
    brain_data_filepath,
    train_test_markers_filepath,
    subjs_to_use = None,
    response_transform_func = trialtype_resp_trans_func,
    clean=None)

all_subjects['groups']

subj_list = np.unique(all_subjects['groups'])

del all_subjects
gc.collect()

checked for intersection and no intersection between the brain data and the subjects was found.
there were 58 subjects overlapping between the subjects marked for train data and the training dump file itself.
test_train_set: 9549
pkl_file: 168
brain_data_filepath: 152
train_test_markers_filepath: 141
response_transform_func: 136
sys: 72
Brain_Data_allsubs: 48
clean: 16
subjs_to_use: 16


/gpfs/projects/sanlab/shared/DEV/DEV_scripts/fMRI/ml/apply_loocv_and_save.py:209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Brain_Data_allsubs.Y[Brain_Data_allsubs.Y=='NULL']=None


9037
9037


0

In [9]:
summary_results = get_all_subjs_discriminability_masked(
    subj_list, "v_1_pfc",ml_data_folderpath=ml_data_folderpath,
    subject_discrim_args = 
    {'train_test_markers_filepath':train_test_markers_filepath, 
     'brain_data_filepath':brain_data_filepath, 
     'resp_trans_func': trialtype_resp_trans_func,
     'mask':pfc_mask})

DEV005

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml/SST/discriminability_cache/v_1_pfc/dtt_results_DEV005.pkl


pre-loaded.

DEV006

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml/SST/discriminability_cache/v_1_pfc/dtt_results_DEV006.pkl


...

correct-go      85
correct-stop    20
Name: trial_type, dtype: int64

setting up decoder...

fitting

evaluating

{'mean_cv_scores': 0.4000000000000001,
 'overfit_accuracy': 0.7047619047619048,
 'overfit_y_pred_vs_obs':              y_obs        y_pred
 8932    correct-go    correct-go
 8933    correct-go    correct-go
 8934    correct-go  correct-stop
 8935    correct-go    correct-go
 8936    correct-go    correct-go
 ...            ...           ...
 9032    correct-go    correct-go
 9033    correct-go  correct-stop
 9034    correct-go  correct-stop
 9035    correct-go    correct-go
 9036  correct-stop  correct-stop
 
 [105 rows x 2 columns]}

## RTFS